In [1]:
from datasets import load_dataset
from dotenv import load_dotenv
import os
from huggingface_hub import login
import pandas as pd
from generator_flow import generator
from generator_flow import normal_generator

Load examples

In [2]:
load_dotenv()
login(os.getenv("HF_API_KEY"))
dataset = load_dataset("Tamiza/zimpl_data")
df_manual = dataset['test'].to_pandas()
dataset

README.md:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

manual_test.json:   0%|          | 0.00/101k [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input', 'output', 'parameter', 'solution'],
        num_rows: 67
    })
})

Table for manual tests

In [3]:
df_manual


,input,output,parameter,solution
0,A bakery produces two types of cookies: chocol...,"set CookieTypes := {""Caramel"", ""ChocolateChip""...",1,objective value: ...
1,"""The factory produces two products, W1 and W2,...","set Products := {""W1"", ""W2""};\nset Resources :...",1,objective value: ...
2,A mining company produces lignite and anthraci...,"set CoalTypes := {""Lignite"", ""Anthracite""};\ns...",1,objective value: ...
3,"A company has two warehouses, Warehouse A and ...","set Warehouses := {""A"", ""B""};\nset Stores := {...",1,objective value: ...
4,A nutritionist wants to create a diet plan for...,"set Foods := {""Food1"", ""Food2"", ""Food3""};\nset...",1,objective value: ...
...,...,...,...,...
62,A health-conscious family wants to have a very...,var x; # number of banana servings taken\nvar ...,0,objective value: 11.363636...
63,A campsite for caravans and tents has an area ...,var x integer; # number of caravans\nvar y ...,0,objective value: ...
64,A factory manufactures doodads and whirligigs....,var x integer; # Each doodad costs $2 to make ...,0,objective value: ...
65,The annual subscription for a tennis club is £...,var x integer; # number of adult members\nv...,0,objective value: ...


Generate responses using AI ZIMPL Generator (for now first 10)

In [4]:
df_manual['ai_advanced'] = df_manual.apply(lambda row: generator(row['input'], row['parameter']), axis=1)

Show answers

In [5]:
df_manual[:5].map(lambda x: str(x)).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

,input,output,parameter,solution,ai_advanced
0,"A bakery produces two types of cookies: chocolate chip and caramel. The bakery anticipates daily demand for a minimum of 80 caramelized & 120 chocolate chip cookies. Due to a lack of raw materials and labor, the bakery can produce 120 caramel cookies and 140 chocolate chip cookies daily. For the bakery to be viable, it must sell a minimum of 240 cookies each day. Every chocolate chip cookie served generates $0.75 in profit, whereas each caramel biscuit generates $0.88. The solution to the number of chocolate chip and caramel cookies that the bakery must produce each day to maximize profit may be determined using linear programming.","set CookieTypes := {""Caramel"", ""ChocolateChip""}; # Parameters for daily demand and production limits param min_demand[CookieTypes] := <""Caramel""> 80, <""ChocolateChip""> 120; param max_production[CookieTypes] := <""Caramel""> 120, <""ChocolateChip""> 140; # Parameter for minimum total sales param min_total_sales := 240; # Parameters for profit per cookie (in dollars) param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables: quantity of each cookie type produced daily var x[ in CookieTypes] >= min_demand[c]; # Objective function: Maximize total profit maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints: Ensure minimum total cookies are sold subto total_sales: sum in CookieTypes: x[c] >= min_total_sales; # Constraints: Do not exceed daily production limits subto production_limits: forall in CookieTypes do x[c] <= max_production[c];",1,objective value: 210.6 x$Caramel 120 (obj:0.88) x$ChocolateChip 140 (obj:0.75),"# Sets set CookieTypes := {""Caramel"", ""ChocolateChip""}; set Demand := {""MinCaramel"", ""MinChocolateChip"", ""MinTotalCookies""}; set ProductionLimits := {""MaxCaramel"", ""MaxChocolateChip""}; param min_demand[Demand] := <""MinCaramel""> 80, <""MinChocolateChip""> 120, <""MinTotalCookies""> 240; param max_production[ProductionLimits] := <""MaxCaramel""> 120, <""MaxChocolateChip""> 140; param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables var x[ in CookieTypes] integer >= 0; # Quantity of each type of cookie produced daily # Objective function maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints # Ensure minimum demand for each type of cookie subto min_demand_constraints: forall in Demand do if d == ""MinCaramel"" then x[""Caramel""] >= min_demand[d]; else if d == ""MinChocolateChip"" then x[""ChocolateChip""] >= min_demand[d]; else if d == ""MinTotalCookies"" then sum in CookieTypes: x[c] >= min_demand[d]; # Do not exceed daily production limits subto production_limits: forall in ProductionLimits do if p == ""MaxCaramel"" then x[""Caramel""] <= max_production[p]; else if p == ""MaxChocolateChip"" then x[""ChocolateChip""] <= max_production[p];"
1,"""The factory produces two products, W1 and W2, using resources S1, S2, and S3. To produce W1, the factory uses 2 units of S1, 1 unit of S2, and 4 units of S3. To produce W2, the factory uses 2 units of S1 and 2 units of S2. The weekly limit for resources is 14, 8, and 16 units, respectively. The selling prices of the products are $2 and $3, respectively. Determine the production plan that would maximize the profit from sales.""","set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; # Parameters for resource usage per product param resource_usage[Products * Resources] := | ""S1"", ""S2"", ""S3"" | |""W1"" | 2, 1, 4 | |""W2"" | 2, 2, 0 |; # Weekly resource limits param resource_limits[Resources] := <""S1""> 14, <""S2""> 8, <""S3""> 16; # Selling price per unit of each product param selling_price[Products] := <""W1""> 2, <""W2""> 3; # Decision variables: weekly production quantity of each product var x[ in Products] >= 0; # Objective function: Maximize total profit maximize total_profit: sum in Products: selling_price[p] * x[p]; # Constraints

In [6]:
df_manual['ai_normal'] = df_manual['input'].apply(normal_generator)

In [7]:
from pandas import DataFrame
df_manual[:5][['output', 'ai_advanced', 'ai_normal']].map(lambda x: str(x)).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

,output,ai_advanced,ai_normal
0,"set CookieTypes := {""Caramel"", ""ChocolateChip""}; # Parameters for daily demand and production limits param min_demand[CookieTypes] := <""Caramel""> 80, <""ChocolateChip""> 120; param max_production[CookieTypes] := <""Caramel""> 120, <""ChocolateChip""> 140; # Parameter for minimum total sales param min_total_sales := 240; # Parameters for profit per cookie (in dollars) param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables: quantity of each cookie type produced daily var x[ in CookieTypes] >= min_demand[c]; # Objective function: Maximize total profit maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints: Ensure minimum total cookies are sold subto total_sales: sum in CookieTypes: x[c] >= min_total_sales; # Constraints: Do not exceed daily production limits subto production_limits: forall in CookieTypes do x[c] <= max_production[c];","# Sets set CookieTypes := {""Caramel"", ""ChocolateChip""}; set Demand := {""MinCaramel"", ""MinChocolateChip"", ""MinTotalCookies""}; set ProductionLimits := {""MaxCaramel"", ""MaxChocolateChip""}; param min_demand[Demand] := <""MinCaramel""> 80, <""MinChocolateChip""> 120, <""MinTotalCookies""> 240; param max_production[ProductionLimits] := <""MaxCaramel""> 120, <""MaxChocolateChip""> 140; param profit[CookieTypes] := <""Caramel""> 0.88, <""ChocolateChip""> 0.75; # Decision variables var x[ in CookieTypes] integer >= 0; # Quantity of each type of cookie produced daily # Objective function maximize total_profit: sum in CookieTypes: profit[c] * x[c]; # Constraints # Ensure minimum demand for each type of cookie subto min_demand_constraints: forall in Demand do if d == ""MinCaramel"" then x[""Caramel""] >= min_demand[d]; else if d == ""MinChocolateChip"" then x[""ChocolateChip""] >= min_demand[d]; else if d == ""MinTotalCookies"" then sum in CookieTypes: x[c] >= min_demand[d]; # Do not exceed daily production limits subto production_limits: forall in ProductionLimits do if p == ""MaxCaramel"" then x[""Caramel""] <= max_production[p]; else if p == ""MaxChocolateChip"" then x[""ChocolateChip""] <= max_production[p];",var c: int; # number of chocolate chip cookies produced var r: int; # number of caramel cookies produced # Objective: Maximize profit maximize profit: 0.75 * c + 0.88 * r; # Constraints s.t. demand_chocolate: c >= 120; # Minimum demand for chocolate chip cookies s.t. demand_caramel: r >= 80; # Minimum demand for caramel cookies s.t. production_chocolate: c <= 140; # Maximum production for chocolate chip cookies s.t. production_caramel: r <= 120; # Maximum production for caramel cookies s.t. total_cookies: c + r >= 240; # Minimum total cookies produced # Non-negativity constraints s.t. non_negativity: c >= 0; s.t. non_negativity_r: r >= 0;
1,"set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; # Parameters for resource usage per product param resource_usage[Products * Resources] := | ""S1"", ""S2"", ""S3"" | |""W1"" | 2, 1, 4 | |""W2"" | 2, 2, 0 |; # Weekly resource limits param resource_limits[Resources] := <""S1""> 14, <""S2""> 8, <""S3""> 16; # Selling price per unit of each product param selling_price[Products] := <""W1""> 2, <""W2""> 3; # Decision variables: weekly production quantity of each product var x[ in Products] >= 0; # Objective function: Maximize total profit maximize total_profit: sum in Products: selling_price[p] * x[p]; # Constraints: Ensure resource usage does not exceed weekly limits subto resource_constraints: forall in Resources do sum in Products: resource_usage[p, r] * x[p] <= resource_limits[r];","# Sets set Products := {""W1"", ""W2""}; set Resources := {""S1"", ""S2"", ""S3""}; param resource_limit[Resources] := <""S1""> 14, <""S2""> 8, <""S3""> 16; param requirement[Products * Resources] := | ""S1"", ""S2"", ""S3"" | |""W1"" | 2, 1, 4 | |""W2"" | 2, 2, 0 |; param price[Products] := <""W1""> 2, <""W2""> 3; # Decision variables var x[ in Pro

## Solver results

In [8]:
from solver import run_scip_and_extract_solution
SCIP_PATH = r"C:\\Program Files\\SCIPOptSuite 9.1.0\\bin"

df_manual["ai_advanced"] = df_manual["ai_advanced"].astype(str)
df_manual["ai_normal"] = df_manual["ai_normal"].astype(str)

def process_zimpl_code(zimpl_code):
    try:
        valid, result = run_scip_and_extract_solution(zimpl_code, SCIP_PATH)
        print(f"Valid: {valid}, Result: {result}")
        return valid, result
    except Exception as e:
        print(f"Error processing code: {e}")
        return False, None



df_manual[["valid_advanced", "result_advanced"]] = df_manual["ai_advanced"].apply(
    lambda code: pd.Series(process_zimpl_code(code))
)

df_manual[["valid_normal", "result_normal"]] = df_manual["ai_normal"].apply(
    lambda code: pd.Series(process_zimpl_code(code))
)

Is Valid: True

Solution:
objective value:                                210.6
x$Caramel                                         120 	(obj:0.88)
x$ChocolateChip                                   140 	(obj:0.75)
Valid: False, Result: No solution found
Valid: True, Result: objective value:                                   14
x$W1                                                4 	(obj:2)
x$W2                                                2 	(obj:3)
Valid: True, Result: objective value:                                   10
x$Lignite                                           1 	(obj:4)
x$Anthracite                                        2 	(obj:3)
Valid: False, Result: No solution found
Valid: True, Result: objective value:                                   12
x$Food1                                             6 	(obj:2)
Valid: True, Result: objective value:                                 5305
x$ProductA                                        124 	(obj:20)
x$ProductB                   

## Compare validation

In [9]:
import pandas as pd

print("ZIMPL generator results")
comparison_table = pd.DataFrame({
    "Category": [
        "Valid (True)",
        "Not Valid (False)",
    ],
    "Count": [
        (df_manual["valid_advanced"] == True).sum(),
        (df_manual["valid_advanced"] == False).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

print("ZIMPL generator results - hardcoded")
comparison_table = pd.DataFrame({
    "Category": [
        "Valid hardcoded (True)",
        "Not Valid hardcoded (False)",
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["parameter"] == 0)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["parameter"] == 0)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

print("ZIMPL generator results - params")
comparison_table = pd.DataFrame({
    "Category": [
        "Valid with params (True)",
        "Not Valid with params (False)"
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["parameter"] == 1)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["parameter"] == 1)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)

print("Normal OpenAI model results")
comparison_table_normal = pd.DataFrame({
    "Category": [
        "Valid (True)",
        "Not Valid (False)",
    ],
    "Count": [
        (df_manual["valid_normal"] == True).sum(),
        (df_manual["valid_normal"] == False).sum(),
    ]
})

comparison_table_normal["Percentage"] = (
    comparison_table_normal["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table_normal)

print("All results")
comparison_table = pd.DataFrame({
    "Category": [
        "Both Valid (True, True)",
        "Advanced Valid Only (True, False)",
        "Normal Valid Only (False, True)",
        "Both Invalid (False, False)",
    ],
    "Count": [
        ((df_manual["valid_advanced"] == True) & (df_manual["valid_normal"] == True)).sum(),
        ((df_manual["valid_advanced"] == True) & (df_manual["valid_normal"] == False)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["valid_normal"] == True)).sum(),
        ((df_manual["valid_advanced"] == False) & (df_manual["valid_normal"] == False)).sum(),
    ]
})

comparison_table["Percentage"] = (
    comparison_table["Count"] / len(df_manual) * 100
).round(2)

print(comparison_table)


ZIMPL generator results
            Category  Count  Percentage
0       Valid (True)     50       74.63
1  Not Valid (False)     17       25.37
ZIMPL generator results - hardcoded
                      Category  Count  Percentage
0       Valid hardcoded (True)     26       38.81
1  Not Valid hardcoded (False)      8       11.94
ZIMPL generator results - params
                        Category  Count  Percentage
0       Valid with params (True)     24       35.82
1  Not Valid with params (False)      9       13.43
Normal OpenAI model results
            Category  Count  Percentage
0       Valid (True)      0         0.0
1  Not Valid (False)     67       100.0
All results
                            Category  Count  Percentage
0            Both Valid (True, True)      0        0.00
1  Advanced Valid Only (True, False)     50       74.63
2    Normal Valid Only (False, True)      0        0.00
3        Both Invalid (False, False)     17       25.37


In [23]:
import pandas as pd

# Filtruj wiersze, gdzie valid_advanced == False
notvalid_rows_advanced = df_manual[df_manual['valid_advanced'] == False]

# Wybierz kolumny 'output', 'ai_advanced', 'ai_normal'
selected_columns = notvalid_rows_advanced[['output', 'ai_advanced', 'ai_normal']]

# Zapisz dane do pliku Excel (XLSX)
selected_columns.to_excel('not_valid.xlsx', index=False, engine='openpyxl')

print("Dane zostały zapisane w formacie XLSX do pliku not_valid.xlsx.")


Dane zostały zapisane w formacie XLSX do pliku not_valid.xlsx.


In [18]:
from IPython.display import display, HTML

df_manual["ai_advanced_display"] = df_manual["ai_advanced"].str.replace('\n', '<br>', regex=False)

html_content = df_manual["ai_advanced_display"].apply(lambda x: f'<div>{x}</div>').to_list()
#display(HTML(''.join(html_content)))

Check valid rows and search for correct results in advanced ZIMPL creator

In [16]:
from obj_fun_validator import compare_solutions

valid_rows_advanced = df_manual[df_manual['valid_advanced'] == True]
notvalid_rows_advanced = df_manual[df_manual['valid_advanced'] == False]
#print(valid_rows_advanced["result_advanced"])
#print(notvalid_rows_advanced["result_advanced"])

df_manual.loc[valid_rows_advanced.index, 'compare_advanced'] = valid_rows_advanced.apply(
    lambda row: compare_solutions(row['solution'], row['result_advanced'])[0], axis=1
)

valid_count = len(valid_rows_advanced)

print(f"valid_advanced == True: {valid_count}")
print(df_manual['compare_advanced'])

total_count = len(df_manual)
nan_count = df_manual['compare_advanced'].isna().sum()
true_count = (df_manual['compare_advanced'] == True).sum()
false_count = (df_manual['compare_advanced'] == False).sum()

nan_percentage = (nan_count / total_count) * 100
true_percentage = (true_count / total_count) * 100
false_percentage = (false_count / total_count) * 100

print(f"All stats:")
print(f"NaN: {nan_count} ({nan_percentage:.2f}%)")
print(f"True: {true_count} ({true_percentage:.2f}%)")
print(f"False: {false_count} ({false_percentage:.2f}%)")

non_nan_rows = df_manual.loc[~df_manual['compare_advanced'].isna()]
true_non_nan_count = (non_nan_rows['compare_advanced'] == True).sum()
false_non_nan_count = (non_nan_rows['compare_advanced'] == False).sum()

true_non_nan_percentage = (true_non_nan_count / valid_count) * 100
false_non_nan_percentage = (false_non_nan_count / valid_count) * 100

print(f"\nStats without NaN (valid_count = {valid_count}):")
print(f"True: {true_non_nan_count} ({true_non_nan_percentage:.2f}%)")
print(f"False: {false_non_nan_count} ({false_non_nan_percentage:.2f}%)")

notvalid_rows_advanced["ai_advanced"].head(10)


valid_advanced == True: 50
0       NaN
1      True
2      True
3       NaN
4      True
      ...  
62      NaN
63      NaN
64    False
65     True
66     True
Name: compare_advanced, Length: 67, dtype: object
All stats:
NaN: 17 (25.37%)
True: 42 (62.69%)
False: 8 (11.94%)

Stats without NaN (valid_count = 50):
True: 42 (84.00%)
False: 8 (16.00%)


0     # Sets\nset CookieTypes := {"Caramel", "Chocol...
3     # Sets\nset Warehouses := {"WarehouseA", "Ware...
20    # Sets\nset BusTypes := {"LargeBus", "SmallBus...
21    # Sets\nset Offers := {"A", "B"};\nset Product...
22    # Sets\nset ScrewTypes := {"WoodScrews", "Meta...
24    # Sets\nset Depots      := {"X", "Y"};\nset Pu...
25    # Sets\nset WorkerTypes := {"Unskilled", "Skil...
26    # Sets\nset Crops := {"Barley", "Swedes"};\nse...
27    # Sets\nset Fruits       := {"Apples", "Banana...
52    # Decision variables\nvar b; # number of blous...
Name: ai_advanced, dtype: object

Check valid rows and search for correct results in normal GPT model

In [ ]:
valid_rows_normal = df_manual[df_manual['valid_normal'] == True]
#print(valid_rows["result_normal"])

df_manual.loc[valid_rows_normal.index, 'compare_normal'] = valid_rows_normal.apply(
    lambda row: compare_solutions(row['solution'], row['result_normal'])[0], axis=1
)

valid_count = len(valid_rows_normal)

print(f"valid_normal == True: {valid_count}")
print(df_manual['compare_normal'])

total_count = len(df_manual)
nan_count = df_manual['compare_normal'].isna().sum()
true_count = (df_manual['compare_normal'] == True).sum()
false_count = (df_manual['compare_normal'] == False).sum()

nan_percentage = (nan_count / total_count) * 100
true_percentage = (true_count / total_count) * 100
false_percentage = (false_count / total_count) * 100

print(f"All stats:")
print(f"NaN: {nan_count} ({nan_percentage:.2f}%)")
print(f"True: {true_count} ({true_percentage:.2f}%)")
print(f"False: {false_count} ({false_percentage:.2f}%)")

non_nan_rows = df_manual.loc[~df_manual['compare_normal'].isna()]
true_non_nan_count = (non_nan_rows['compare_normal'] == True).sum()
false_non_nan_count = (non_nan_rows['compare_normal'] == False).sum()

true_non_nan_percentage = (true_non_nan_count / valid_count) * 100
false_non_nan_percentage = (false_non_nan_count / valid_count) * 100

print(f"\nStats without NaN (valid_count = {valid_count}):")
print(f"True: {true_non_nan_count} ({true_non_nan_percentage:.2f}%)")
print(f"False: {false_non_nan_count} ({false_non_nan_percentage:.2f}%)")

valid_normal == True: 0
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
62    NaN
63    NaN
64    NaN
65    NaN
66    NaN
Name: compare_normal, Length: 67, dtype: object
All stats:
NaN: 67 (100.00%)
True: 0 (0.00%)
False: 0 (0.00%)

Stats without NaN (valid_count = 0):
True: 0 (nan%)
False: 0 (nan%)


C:\Users\jagod\AppData\Local\Temp\ipykernel_19616\3424765584.py:31: RuntimeWarning: invalid value encountered in scalar divide
  true_non_nan_percentage = (true_non_nan_count / valid_count) * 100
C:\Users\jagod\AppData\Local\Temp\ipykernel_19616\3424765584.py:32: RuntimeWarning: invalid value encountered in scalar divide
  false_non_nan_percentage = (false_non_nan_count / valid_count) * 100


In [ ]:
from generator_flow import validate_generator

validate_generator(df_manual['output'], df_manual['ai_advanced'])

ImportError: cannot import name 'validate_generator' from 'generator_flow' (c:\Users\jagod\git\zimpl_ai_generator\generator_flow.py)

TODO


- weryfikacja kodu poprzez: punktowanie przez niezależy model językowy (kolumna ai_advanced)